In [1]:
import sys
import os
sys.path.append(os.getcwd()+'/fivepoint')
import build.fivep as f
import time
import torch
import torch.nn.functional as F
from spherical_distortion.functional import create_tangent_images, unresample
from spherical_distortion.util import *
import matplotlib.pyplot as plt
from skimage import io
import os
import cv2

import sys
import pandas as pd
import numpy as np
import _spherical_distortion_ext._mesh as _mesh
import argparse

from random import sample
import imageio
from scipy.spatial.transform import Rotation as Rot

from utils.coord    import coord_3d
from utils.ransac   import *
from utils.keypoint import *
from utils.metrics  import *
from utils.camera_recovering import *

from os import listdir
from os.path import isfile, join, isdir
from tqdm import tqdm

sys.path.append(os.getcwd()+'/SPHORB-master')

import build1.sphorb_cpp as sphorb

/home/imonalc/anaconda3/envs/keypoints/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sort_key(pts1, pts2, desc1, desc2, points):

    ind1 = np.argsort(pts1[:,2].numpy(),axis = 0)[::-1]
    ind2 = np.argsort(pts2[:,2].numpy(),axis = 0)[::-1]

    max1 = np.min([points,ind1.shape[0]])
    max2 = np.min([points,ind2.shape[0]])

    ind1 = ind1[:max1]
    ind2 = ind2[:max2]

    pts1 = pts1[ind1.copy(),:]
    pts2 = pts2[ind2.copy(),:]

    desc1 = desc1[:,ind1.copy()]
    desc2 = desc2[:,ind2.copy()]

    pts1 = np.concatenate((pts1[:,:2], np.ones((pts1.shape[0],1))), axis = 1 )
    pts2 = np.concatenate((pts2[:,:2], np.ones((pts2.shape[0],1))), axis = 1 )

    desc1 = np.transpose(desc1,[1,0]).numpy()
    desc2 = np.transpose(desc2,[1,0]).numpy()

    return pts1, pts2, desc1, desc2


def mnn_matcher(desc1, desc2, method="mean_std"):
    sim = desc1 @ desc2.transpose()
    if method == "mean_std":
        k = 4
        threshold = sim.mean() + k * sim.std()
    sim_raw = sim.copy()
    sim[sim < threshold] = 0
    nn12 = np.argmax(sim, axis=1)
    nn21 = np.argmax(sim, axis=0)
    ids1 = np.arange(0, sim.shape[0])
    mask = (ids1 == nn21[nn12])
    matches = np.stack([ids1[mask], nn12[mask]])
    return matches.transpose(), sim_raw


def euclid_matcher(desc1, desc2, method="mean_std"):
    dists = np.sum((desc1[:, np.newaxis] - desc2)**2, axis=2)

    if method == "mean_std":
        k = 4
        threshold = dists.mean() + k * dists.std()
    
    dists_raw = dists.copy()
    dists[dists > threshold] = np.inf
    nn12 = np.argmin(dists, axis=1)
    nn21 = np.argmin(dists, axis=0)
    ids1 = np.arange(0, dists.shape[0])
    mask = (ids1 == nn21[nn12])
    matches = np.stack([ids1[mask], nn12[mask]])
    return matches.transpose(), dists_raw


def matched_points(pts1, pts2, desc1, desc2, opt, args_opt, match='ratio', use_new_method=0):
    if opt[-1] == 'p':
        porce = int(opt[:-1])
        n_key = int(porce/100 * pts1.shape[0])
    else:
        n_key = int(opt)

    s_pts1  = pts1.copy()[:n_key,:]
    s_pts2  = pts2.copy()[:n_key,:]
    s_desc1 = desc1.copy().astype('float32')[:n_key,:]
    s_desc2 = desc2.copy().astype('float32')[:n_key,:]

    if 'orb' in args_opt:
        s_desc1 = s_desc1.astype(np.uint8)
        s_desc2 = s_desc2.astype(np.uint8)
        hamming_distances = np.array([[cv2.norm(d1, d2, cv2.NORM_HAMMING) for d2 in s_desc2] for d1 in s_desc1])
        sim = -hamming_distances
        nn12 = np.argmin(hamming_distances, axis=1)
        nn21 = np.argmin(hamming_distances, axis=0)
        ids1 = np.arange(len(s_desc1))
        mask = (ids1 == nn21[nn12])
        matches = [cv2.DMatch(i, j, 0) for i, j in zip(ids1[mask], nn12[mask])]
    elif "superpoint" in args_opt:
        matches_idx, sim = mnn_matcher(s_desc1, s_desc2)
        matches = [cv2.DMatch(i, j, 0) for i, j in matches_idx]
    elif "sift" in args_opt:
        matches_idx, sim = euclid_matcher(s_desc1, s_desc2)
        matches = [cv2.DMatch(i, j, 0) for i, j in matches_idx]


    M = np.zeros((2,len(matches)))
    for ind, match in zip(np.arange(len(matches)),matches):
        M[0,ind] = match.queryIdx
        M[1,ind] = match.trainIdx


    return s_pts1, s_pts2, s_pts1[M[0,:].astype(int),:3], s_pts2[M[1,:].astype(int),:3], sim


def get_error(x1, x2, Rx, Tx):

    S = computeEssentialMatrixByRANSAC(x1, x2)
    I = S[1]
    I = I.astype(np.int64)

    x1 = x1[I,:]
    x2 = x2[I,:]

    F = calc_ematrix(x1,x2)


    R1,R2,T1,T2 = decomposeE(F)

    R,T = choose_rt(R1,R2,T1,T2,x1,x2)

    R_error = r_error(Rx,R)
    T_error = t_error(Tx,T)

    return R_error, T_error


def get_descriptor(descriptor):
    if descriptor == 'sphorb':
        return 'sphorb', 'erp', 640, 0
    elif descriptor == 'sift':
        return 'sift', 'erp', 512, 0
    elif descriptor == 'tsift':
        return 'sift', 'tangent', 512, 0
    elif descriptor == 'orb':
        return 'orb', 'erp', 512, 0
    elif descriptor == 'torb':
        return 'orb', 'tangent', 512, 0
    elif descriptor == 'spoint':
        return 'superpoint', 'erp', 512, 0
    elif descriptor == 'tspoint':
        return 'superpoint', 'tangent', 512, 0
    elif descriptor == 'alike':
        return 'alike', 'erp', 512, 0
    elif descriptor == 'talike':
        return 'alike', 'tangent', 512, 0
    elif descriptor == 'Proposed':
        return 'superpoint', 'tangent', 512, 1
    elif descriptor == 'Ltspoint':
        return 'superpoint', 'tangent', 512, 2
    elif descriptor == 'Ftspoint':
        return 'superpoint', 'tangent', 512, 3


def get_error(x1, x2, Rx, Tx):

    S = computeEssentialMatrixByRANSAC(x1, x2)
    I = S[1]
    I = I.astype(np.int64)

    x1 = x1[I,:]
    x2 = x2[I,:]

    F = calc_ematrix(x1,x2)


    R1,R2,T1,T2 = decomposeE(F)

    R,T = choose_rt(R1,R2,T1,T2,x1,x2)

    R_error = r_error(Rx,R)
    T_error = t_error(Tx,T)

    return R_error, T_error


def AUC(ROT, TRA, MET, L):

    RAUC  = np.zeros(len(L))
    TAUC  = np.zeros(len(L))

    for index, t in enumerate(L):
        ids = np.where(ROT<np.radians(t))[0]
        RAUC[index] = len(ids)/len(ROT)

    for index, t in enumerate(L):
        ids = np.where(TRA<np.radians(t))[0]
        TAUC[index] = len(ids)/len(TRA)

    return RAUC, TAUC, np.array(MET)


def get_data(DATAS):
    if len(DATAS) == 1:
        data = DATAS[0]
    elif set(['Urban1','Urban2','Urban3','Urban4']) == set(DATAS):
        data = 'Outdoor'
    elif set(['Realistic','Interior1','Interior2','Room','Classroom']) == set(DATAS):
        data = 'Indoor'
    elif set(['Urban1_R','Urban2_R','Urban3_R','Urban4_R','Realistic_R','Interior1_R','Interior2_R','Room_R','Classroom_R']) == set(DATAS):
        data = 'OnlyRot'
    elif set(['Urban1_T','Urban2_T','Urban3_T','Urban4_T','Realistic_T','Interior1_T','Interior2_T','Room_T','Classroom_T']) == set(DATAS):
        data = 'OnlyTra'
    else:
        data = ''
        for DA in DATAS:
            data+=DA

    return data


def get_kd(array):

    array = np.array(array)
    delimiter = int(array[-1])
    A = array[:-1]
    K = A[:delimiter].reshape(-1,3)
    D = A[delimiter:].reshape(-1,32)
    return K,D


def normalize_features(features):
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    normalized_features = features / norms

    threshold = 0.2
    normalized_features = np.minimum(normalized_features, threshold)

    norms = np.linalg.norm(normalized_features, axis=1, keepdims=True)
    normalized_features /= norms

    return normalized_features

In [3]:
def test(points = 500,
match = "ratio",
descriptor = "tsift",
path = "./data/Farm_pair",):
    t0 = time.time()
    descriptor = descriptor

    opt, mode, sphered, use_our_method = get_descriptor(descriptor)
    base_order = 0  # Base sphere resolution
    sample_order = 8  # Determines sample resolution (10 = 2048 x 4096)
    scale_factor = 1.0  # How much to scale input equirectangular image by
    save_ply = False  # Whether to save the PLY visualizations too
    dim = np.array([2*sphered, sphered])

    path_o = path + '/O.png'
    path_r = path + '/R.png'
    img_o = load_torch_img(path_o)[:3, ...].float()
    img_o = F.interpolate(img_o.unsqueeze(0), scale_factor=scale_factor, mode='bilinear', align_corners=False, recompute_scale_factor=True).squeeze(0)
    img_r = load_torch_img(path_r)[:3, ...].float()
    img_r = F.interpolate(img_r.unsqueeze(0), scale_factor=scale_factor, mode='bilinear', align_corners=False, recompute_scale_factor=True).squeeze(0)
    img_o = torch2numpy(img_o.byte())
    img_r = torch2numpy(img_r.byte())
    img_o = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
    img_r = cv2.cvtColor(img_r, cv2.COLOR_BGR2RGB)
    height_threshold = 0.7 * img_o.shape[0]


    print(path_o)
    t1 = time.time()
    print("image:", t1-t0)
    if opt != 'sphorb':
        corners = tangent_image_corners(base_order, sample_order)
        pts1, desc1 = process_image_to_keypoints(path_o, corners, scale_factor, base_order, sample_order, opt, mode)
        print(descriptor, len(pts1))
        pts2, desc2 = process_image_to_keypoints(path_r, corners, scale_factor, base_order, sample_order, opt, mode)
        pts1[pts1[:,0] > img_o.shape[1], 0] -= img_o.shape[1]
        pts2[pts2[:,0] > img_o.shape[1], 0] -= img_o.shape[1]
    else:      
        os.chdir('SPHORB-master/')
        pts1, desc1 = get_kd(sphorb.sphorb(path_o, points))
        pts2, desc2 = get_kd(sphorb.sphorb(path_r, points))
        os.chdir('../')
    #if opt == "sift":
    #    desc1 = normalize_features(desc1)
    #    desc2 = normalize_features(desc2)

    #print(desc1.shape, desc1)
    pts1, pts2, desc1, desc2, _, _ = sort_key(pts1, pts2, desc1, desc2, points)
    t2 = time.time()
    print("detection:", t2-t1)
    if len(pts1.shape) == 1:
        pts1 = pts1.reshape(1,-1)

    print(desc1.shape)
    s_pts1, s_pts2, x1, x2, sim = matched_points(pts1, pts2, desc1, desc2, "100p", opt, match, use_new_method=use_our_method)
    return sim

In [4]:
sim_p = test(descriptor="Proposed")
sim_o = test(descriptor="orb")
sim_to = test(descriptor="torb")
sim_s = test(descriptor="sift")
sim_ts = test(descriptor="tsift")
sim_sp = test(descriptor="spoint")


./data/Farm_pair/O.png
image: 0.12318563461303711


/home/imonalc/anaconda3/envs/keypoints/lib/python3.7/site-packages/torch/nn/functional.py:4228: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


Proposed 520
detection: 1.9257798194885254
(500, 256)
./data/Farm_pair/O.png
image: 0.1635584831237793
orb 9263
detection: 0.41322803497314453
(500, 32)
./data/Farm_pair/O.png
image: 0.03667473793029785
torb 10480
detection: 0.4716174602508545
(500, 32)
./data/Farm_pair/O.png
image: 0.03555583953857422
sift 3241
detection: 0.5002164840698242
(500, 128)
./data/Farm_pair/O.png
image: 0.0409855842590332
tsift 1859
detection: 2.404430389404297
(500, 128)
./data/Farm_pair/O.png
image: 0.03394937515258789
spoint 1013
detection: 0.4361600875854492
(500, 256)


In [5]:
sim_dict = {
    "spoint": sim_sp,
    "tspoint": sim_p,
    "orb": sim_o,
    "torb": sim_to,
    "sift": sim_s,
    "tsift": sim_ts 
}


In [6]:
import pickle
with open("sim.pickle", mode='wb') as f:
    pickle.dump(sim_dict,f)